# Residual Networks

In theory, very deep networks can represent very complex functions; but in practice, they are hard to train. Residual Networks, introduced by He et al., allow you to train much deeper networks than were previously practically feasible.

In [2]:
import numpy as np
import keras
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import h5py
import json

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

### The problem of very deep neural networks

The main benefit of a very deep network is that it can represent very complex functions. It can also learn features at many different levels of abstraction, from edges (at the lower layers) to very complex features (at the deeper layers). However, using a deeper network doesn't always help. A huge barrier to training them is vanishing gradients: very deep networks often have a gradient signal that goes to zero quickly, thus making gradient descent unbearably slow. More specifically, during gradient descent, as you backprop from the final layer back to the first layer, you are multiplying by the weight matrix on each step, and thus the gradient can decrease exponentially quickly to zero (or, in rare cases, grow exponentially quickly and "explode" to take very large values).

### Building a Residual Network

In ResNets, a "shortcut" or a "skip connection" allows the gradient to be directly backpropagated to earlier layers.

Having ResNet blocks with the shortcut also makes it very easy for one of the blocks to learn an identity function. This means that you can stack on additional ResNet blocks with little risk of harming training set performance. (There is also some evidence that the ease of learning an identity function--even more than skip connections helping with vanishing gradients--accounts for ResNets' remarkable performance.)

Two main types of blocks are used in a ResNet, depending mainly on whether the input/output dimensions are same or different.

    -> The identity block

The identity block is the standard block used in ResNets, and corresponds to the case where the input activation (say $a^{[l]}$) has the same dimension as the output activation (say $z^{[l+2]}$). In other words, the shape of $z^{[l+2]}$ is
matched to that of $a^{[l]}$ using a (1,1) convolution(to match the depth/channels).
    
    -> The Convolutional block
    
This type of block is used when the input and output dimensions don't match up. The difference with the identity block is that there is a CONV2D layer in the shortcut path. CONV2D layer in the shortcut path is used to resize the input $a^{[l]}$ 
to a different dimension, so that the dimensions match up in the final addition to $z^{[l+2]}$, needed to add the shortcut 
value back to the main path.

In [76]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block with skip connection over 3 layers, a[l] to z[l+3]

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network

    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    # First Component - Conv (1,1) + BN along channels + ReLU
    X = keras.layers.Conv2D(filters=F1, 
                            kernel_size=(1,1), 
                            strides=(1,1), 
                            padding='same', 
                            name=conv_name_base+'2a', 
                            kernel_initializer=glorot_uniform(seed=0)
                           )(X)
    X = keras.layers.BatchNormalization(axis=3, name=bn_name_base+'2a')(X)
    X = keras.layers.Activation(activation='relu')(X)
    
    # Second Component - Conv (f,f) + BN along channels + ReLU
    X = keras.layers.Conv2D(filters=F2, 
                            kernel_size=(f,f), 
                            strides=(1,1), 
                            padding='same', 
                            name=conv_name_base+'2b', 
                            kernel_initializer=glorot_uniform(seed=0)
                           )(X)
    X = keras.layers.BatchNormalization(axis=3, name=bn_name_base+'2b')(X)
    X = keras.layers.Activation(activation='relu')(X)
    
    # Third Component - Conv (1,1) + BN along channels + add shortcut(residual) + ReLU
    X = keras.layers.Conv2D(filters=F3, 
                            kernel_size=(1,1), 
                            strides=(1,1), 
                            padding='same', 
                            name=conv_name_base+'2c', 
                            kernel_initializer=glorot_uniform(seed=0)
                           )(X)
    X = keras.layers.BatchNormalization(axis=3, name=bn_name_base+'2c')(X)
    #assert(X.shape == X_shortcut.shape) # Shape matches up in identity block
    X = keras.layers.Add()([X, X_shortcut])
    X = keras.layers.Activation(activation='relu')(X)
    
    return X  
        

In [29]:
tf.reset_default_graph()

with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder("float", [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    A = identity_block(A_prev, f=2, filters=[2, 4, 6], stage=1, block='a')
    test.run(tf.global_variables_initializer())
    out = test.run([A], feed_dict={A_prev: X, K.learning_phase(): 0})
    print("out = " + str(out[0][1][1][0]))
    print("Shape of output: " + str(out[0].shape))

out = [0.94822985 0.         1.1610144  2.747859   0.         1.36677   ]
Shape of output: (3, 4, 4, 6)


In [84]:
def convolutional_block(X, f, filters, stage, block, s=1):
    """
    Implementation of the convolutional block  with skip connection over 3 layers, a[l] to z[l+3] and 
    Conv2D in the shortcut path to match the shapes of a[l] and z[l+3] during addition

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used

    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    # First Component - Conv (1,1) and stride(s,s) + BN along channels + ReLU
    X = keras.layers.Conv2D(filters=F1, 
                            kernel_size=(1,1), 
                            strides=(s,s),
                            padding='same',
                            name=conv_name_base+'2a', 
                            kernel_initializer=glorot_uniform(seed=0)
                           )(X)
    #print('I: '+str(X.shape))
    X = keras.layers.BatchNormalization(axis=3, name=bn_name_base+'2a')(X)
    X = keras.layers.Activation(activation='relu')(X)
    
    # Second Component - Conv (f,f) + BN along channels + ReLU
    X = keras.layers.Conv2D(filters=F2, 
                            kernel_size=(f,f), 
                            strides=(1,1), 
                            padding='same', 
                            name=conv_name_base+'2b', 
                            kernel_initializer=glorot_uniform(seed=0)
                           )(X)
    #print('II: '+str(X.shape))
    X = keras.layers.BatchNormalization(axis=3, name=bn_name_base+'2b')(X)
    X = keras.layers.Activation(activation='relu')(X)
    
    # Third Component part I - Conv (1,1) + BN along channels
    X = keras.layers.Conv2D(filters=F3, 
                            kernel_size=(1,1), 
                            strides=(1,1), 
                            padding='same', 
                            name=conv_name_base+'2c', 
                            kernel_initializer=glorot_uniform(seed=0)
                           )(X)
    #print('III: '+str(X.shape))
    X = keras.layers.BatchNormalization(axis=3, name=bn_name_base+'2c')(X)
    
    # Shortcut Component - F3 Conv (1,1) filters and stride(s,s) + BN along channels
    X_shortcut = keras.layers.Conv2D(filters=F3, 
                                     kernel_size=(1,1), 
                                     strides=(s,s),
                                     padding='same',
                                     name=conv_name_base+'1', 
                                     kernel_initializer=glorot_uniform(seed=0)
                                    )(X_shortcut)
    X_shortcut = keras.layers.BatchNormalization(axis=3, name=bn_name_base+'1')(X_shortcut)
    
    # Third Component part II - Add layer(main and shortcut) + ReLU
    #print("X: " + str(X.shape))
    #print("X_shortcut: " + str(X_shortcut.shape))
    #assert(X.shape == X_shortcut.shape)
    X = keras.layers.Add()([X, X_shortcut])
    X = keras.layers.Activation(activation='relu')(X)
    
    return X


In [85]:
tf.reset_default_graph()

with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder("float", [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    A = convolutional_block(A_prev, f=2, filters=[2, 4, 6], stage=1, block='a')
    test.run(tf.global_variables_initializer())
    out = test.run([A], feed_dict={A_prev: X, K.learning_phase(): 0})
    print("Shape of output: " + str(out[0].shape))
    print("out = " + str(out[0][1][1][0]))

Shape of output: (3, 4, 4, 6)
out = [0.14074254 1.2335387  0.         0.19140035 0.75703484 0.4619543 ]


### Building ResNet - 50

The details of this ResNet-50 model are:
- Zero-padding pads the input with a pad of (3,3)
- Stage 1:
    - The 2D Convolution has 64 filters of shape (7,7) and uses a stride of (2,2). Its name is "conv1".
    - BatchNorm is applied to the channels axis of the input.
    - MaxPooling uses a (3,3) window and a (2,2) stride.
- Stage 2:
    - The convolutional block uses three set of filters of size [64,64,256], "f" is 3, "s" is 1 and the block is "a".
    - The 2 identity blocks use three set of filters of size [64,64,256], "f" is 3 and the blocks are "b" and "c".
- Stage 3:
    - The convolutional block uses three set of filters of size [128,128,512], "f" is 3, "s" is 2 and the block is "a".
    - The 3 identity blocks use three set of filters of size [128,128,512], "f" is 3 and the blocks are "b", "c" and "d".
- Stage 4:
    - The convolutional block uses three set of filters of size [256, 256, 1024], "f" is 3, "s" is 2 and the block is "a".
    - The 5 identity blocks use three set of filters of size [256, 256, 1024], "f" is 3 and the blocks are "b", "c", "d", "e" and "f".
- Stage 5:
    - The convolutional block uses three set of filters of size [512, 512, 2048], "f" is 3, "s" is 2 and the block is "a".
    - The 2 identity blocks use three set of filters of size [512, 512, 2048], "f" is 3 and the blocks are "b" and "c".
- The 2D Average Pooling uses a window of shape (2,2) and its name is "avg_pool".
- The flatten doesn't have any hyperparameters or name.
- The Fully Connected (Dense) layer reduces its input to the number of classes using a softmax activation. Its name should be `'fc' + str(classes)`.

In [80]:
def ResNet50(input_shape=(64, 64, 3), n_classes=6):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    n_classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Create a input layer
    X_input = keras.layers.Input(input_shape)
    
    # Zeropadding (3,3)
    X = keras.layers.ZeroPadding2D((3,3))(X_input)
    
    # Stage I: 64 Conv2D (7,7) filters stride(2,2) + BN along channels + MaxPool (3,3) stride(2,2)
    X = keras.layers.Conv2D(filters=64, 
                            kernel_size=(7,7), 
                            strides=(2,2), 
                            name='conv1', 
                            kernel_initializer=glorot_uniform(seed=0)
                           )(X)
    X = keras.layers.BatchNormalization(axis=3, name='bn_conv1')(X)
    X = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(X)
    
    # Stage II: Conv Block [64,64,256] + 2* Identity Blocks [64,64,256]
    X = convolutional_block(X=X, f=3, filters=[64,64,256], stage=2, block='a', s=1)
    X = identity_block(X=X, f=3, filters=[64,64,256], stage=2, block='b')
    X = identity_block(X=X, f=3, filters=[64,64,256], stage=2, block='c')
    
    # Stage III: Conv Block [128,128,512] + 3* Identity Blocks [128,128,512]
    X = convolutional_block(X=X, f=3, filters=[128,128,512], stage=3, block='a', s=2)
    X = identity_block(X=X, f=3, filters=[128,128,512], stage=3, block='b')
    X = identity_block(X=X, f=3, filters=[128,128,512], stage=3, block='c')
    X = identity_block(X=X, f=3, filters=[128,128,512], stage=3, block='d')
    
    # Stage IV: Conv Block [256,256,1024] + 5* Identity Blocks[256,256,1024]
    X = convolutional_block(X=X, f=3, filters=[256,256,1024], stage=4, block='a', s=2)
    X = identity_block(X=X, f=3, filters=[256,256,1024], stage=4, block='b')
    X = identity_block(X=X, f=3, filters=[256,256,1024], stage=4, block='c')
    X = identity_block(X=X, f=3, filters=[256,256,1024], stage=4, block='d')
    X = identity_block(X=X, f=3, filters=[256,256,1024], stage=4, block='e')
    X = identity_block(X=X, f=3, filters=[256,256,1024], stage=4, block='f')
    
    # Stage V: Conv Block [512,512,2014] + 2* Identity Blocks[512,512,2048]
    X = convolutional_block(X=X, f=3, filters=[512,512,2048], stage=5, block='a', s=2)
    X = identity_block(X=X, f=3, filters=[512,512,2048], stage=5, block='b')
    X = identity_block(X=X, f=3, filters=[512,512,2048], stage=5, block='c')
    
    # Average Pooling
    X = keras.layers.AveragePooling2D(pool_size=(2,2))(X)
    
    # Flatten
    X = keras.layers.Flatten()(X)
    
    # Fully Connected Layer with softmax and n_classes units
    X = keras.layers.Dense(units=n_classes, 
                           activation='softmax', 
                           name='fc'+str(n_classes), 
                           kernel_initializer=glorot_uniform(seed=0)
                          )(X)
    
    # Create an object of Model class from keras.models and pass the input and output
    model = keras.models.Model(inputs=X_input, outputs=X, name='resnet50')
    
    return model   
    

In [88]:
# Create a ResNet50 model object

model = ResNet50(input_shape=(64, 64, 3), n_classes=6)

In [89]:
# Compile the model with adam optimizer, categorical crossentropy loss and accuracy as metrics

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [91]:
# Load the signs dataset

X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig / 255.
X_test = X_test_orig / 255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print("number of training examples = " + str(X_train.shape[0]))
print("number of test examples = " + str(X_test.shape[0]))
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape: " + str(X_test.shape))
print("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


In [104]:
# Run Training of 25 epochs(warning it's a DEEP model and will take 5 to 6 minutes per epoch on CPU)

model.fit(X_train, Y_train, epochs = 25, batch_size = 32)

Epoch 1/3
1080/1080 [==============================] - 305s 282ms/step - loss: 0.6120 - acc: 0.8954
Epoch 2/3
1080/1080 [==============================] - 291s 270ms/step - loss: 0.6376 - acc: 0.8954
Epoch 3/3
1080/1080 [==============================] - 311s 288ms/step - loss: 0.4519 - acc: 0.9481


In [105]:
# Save the model

model.save("ResNet50_hand_signs_25ep.h5")
print("Saved model to disk")

Saved model to disk


In [106]:
# Load the model
model = load_model("ResNet50_hand_signs_25ep.h5")

# Make predictions
preds = model.evaluate(X_test, Y_test)
print("Loss = " + str(preds[0]))
print("Test Accuracy = " + str(preds[1]))

120/120 [==============================] - 17s 145ms/step
Loss = 0.45328784386316934
Test Accuracy = 0.874999996026357


In [107]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 70, 70, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   9472        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 32, 32, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
max_poolin